# 1. Install Packages and Model

In [2]:
!pip install eland elasticsearch
!pip install gradio
!pip install --upgrade urllib3 requests elasticsearch
!pip install sentence_transformers
!pip install --upgrade elasticsearch



In [3]:
from elasticsearch import (Elasticsearch,helpers)
import getpass
import pandas as pd
import urllib3
from urllib.request import urlopen
from sentence_transformers import SentenceTransformer


### BERT Models Installation

In [ ]:
# Load the local model for text embedding
# model1 = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
model = SentenceTransformer("all-mpnet-base-v2", device="cpu")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### Connect to Elastic Cloud

In [ ]:
CLOUD_ID = getpass.getpass("Enter Elastic Cloud ID:  ")           # CLOUD_ID = 20350d848e2d40d796d363d6bd9487d8:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvOjQ0MyRhZDcyYWQ3YTdlZGM0NjdjYjFhMmM4NjQxNWIwMmZmMSQ3ZWM2ODJjMGE4YTM0MjY0YTY1ODBjYWU4MjY0M2IwOQ==
# Password for the 'elastic' user generated by Elasticsearch
ELASTIC_PASSWORD = getpass.getpass("Enter Elastic password:  ")   # ELASTIC_PASSWORD = FyjsNg9xN6LnAo0yFj3gEDTk

# Create the client instance
es = Elasticsearch(cloud_id=CLOUD_ID, basic_auth=("elastic", ELASTIC_PASSWORD), request_timeout=3600)
if es.ping():
  print("You have successfully connected with Elastic Cloud!")
else:
  print("Connection failed. Please check your Cloud ID or credentials.")

Enter Elastic Cloud ID:  ··········
Enter Elastic password:  ··········
You have successfully connected with Elastic Cloud!


# Queries


### - mephone 13 in mumbai
### - macbook wind in mumbai
### - apple pro 13 inch display  
### - apple macbook wind 14 inch display
### - asus gaming laptop
### - 14 inch laptops
### - "کپڑے دھون آلی مشین" - "clothes washing machine"
### - apple 13 small
### - Find all LG AC models with a white color
### - Which refrigerators are single-door models?

# 2. Insurance Data  

## Semantic Searching BERT-mpnet-base-v2   VS  elser-model-2

### 1. BERT-mpnet-base-v2  Model

In [ ]:
# Define the input keyword
input_keyword = "apple 13"




# Compute the query vector locally
query_vector = model.encode(input_keyword)

# Perform the search with the locally computed query vector
response = es.search(
    index="insurance_mpnet_data",  # Updated index name
    size=5,
    knn={
        "field": "ml.embeddings",  # Field storing precomputed embeddings
        "k": 10,  # Number of nearest neighbors to return as top hits.
        "num_candidates": 500,  # Number of nearest neighbor candidates to consider per shard.
        "query_vector": query_vector,  # Use the locally computed query vector
    },
)

# Process and print the results
for hit in response["hits"]["hits"]:
    score = hit["_score"]

    # Extract the required fields from the hit
    customer_location = hit["_source"].get("Customer Location: State", "N/A")
    gadget_model = hit["_source"].get("Gadget Model", "N/A")
    product_description = hit["_source"].get("Product Description", "N/A")
    purchase_price = hit["_source"].get("Purchase Price", "N/A")
    purchase_date = hit["_source"].get("Purchase Date", "N/A")
    current_value = hit["_source"].get("Current Value", "N/A")
    usage_type = hit["_source"].get("Usage Type", "N/A")
    condition = hit["_source"].get("Condition", "N/A")
    coverage_type = hit["_source"].get("Coverage Type", "N/A")
    policy_duration = hit["_source"].get("Policy Duration", "N/A")
    insurance_claim_history = hit["_source"].get("Insurance Claim History", "N/A")
    gadget_age = hit["_source"].get("Gadget Age", "N/A")
    risk_assessment_score = hit["_source"].get("Risk Assessment Score", "N/A")

    print(
        f"\nScore: {score}\n"
        f"Customer Location: {customer_location}\n"
        f"Gadget Model: {gadget_model}\n"
        f"Product Description: {product_description}\n"
        f"Purchase Price: {purchase_price}\n"
        f"Purchase Date: {purchase_date}\n"
        f"Current Value: {current_value}\n"
        f"Usage Type: {usage_type}\n"
        f"Condition: {condition}\n"
        f"Coverage Type: {coverage_type}\n"
        f"Policy Duration: {policy_duration}\n"
        f"Insurance Claim History: {insurance_claim_history}\n"
        f"Gadget Age: {gadget_age}\n"
        f"Risk Assessment Score: {risk_assessment_score}\n"
    )
    print("-"*150)



Score: 0.7598231
Customer Location: Texas
Gadget Model: Ipad Air 13
Product Description: Apple iPad Air 13 with 13-inch Retina display, stylish design, black color, 8GB RAM, 128GB storage
Purchase Price: 3222.39
Purchase Date: 2016-07-20
Current Value: 1334.67
Usage Type: Business
Condition: New
Coverage Type: Comprehensive
Policy Duration: 5
Insurance Claim History: 1
Gadget Age: 8
Risk Assessment Score: 0.21

------------------------------------------------------------------------------------------------------------------------------------------------------

Score: 0.7598231
Customer Location: Texas
Gadget Model: Ipad Air 13
Product Description: Apple iPad Air 13 with 13-inch Retina display, stylish design, black color, 8GB RAM, 128GB storage
Purchase Price: 3597.45
Purchase Date: 2018-11-09
Current Value: 1157.93
Usage Type: Business
Condition: Refurbished
Coverage Type: Comprehensive
Policy Duration: 7
Insurance Claim History: 0
Gadget Age: 6
Risk Assessment Score: 0.89

---------

### 2. elser-model-2 Model

In [ ]:
# Input for the search
input_keyword = "apple 13"

# Search query using Elastic's ELSER model
response = es.search(
    index="insurance-elser2-data",  # Target index name
    size=5,  # Number of documents to retrieve
    body={
        "query": {
            "text_expansion": {
                "combined_text_embedding": {
                    "model_id": ".elser_model_2",  # Model ID for ELSER
                    "model_text": input_keyword  # User query text
                }
            }
        }
    }
)

# Process and print the search results
for hit in response["hits"]["hits"]:
    score = hit["_score"]

    # Extract fields from the hit
    customer_location = hit["_source"].get("Customer Location: State", "N/A")
    gadget_model = hit["_source"].get("Gadget Model", "N/A")
    product_description = hit["_source"].get("Product Description", "N/A")
    purchase_price = hit["_source"].get("Purchase Price", "N/A")
    purchase_date = hit["_source"].get("Purchase Date", "N/A")
    current_value = hit["_source"].get("Current Value", "N/A")
    usage_type = hit["_source"].get("Usage Type", "N/A")
    condition = hit["_source"].get("Condition", "N/A")
    coverage_type = hit["_source"].get("Coverage Type", "N/A")
    policy_duration = hit["_source"].get("Policy Duration", "N/A")
    insurance_claim_history = hit["_source"].get("Insurance Claim History", "N/A")
    gadget_age = hit["_source"].get("Gadget Age", "N/A")
    risk_assessment_score = hit["_source"].get("Risk Assessment Score", "N/A")

    # Print results
    print(
        f"\nScore: {score}\n"
        f"Customer Location: {customer_location}\n"
        f"Gadget Model: {gadget_model}\n"
        f"Product Description: {product_description}\n"
        f"Purchase Price: {purchase_price}\n"
        f"Purchase Date: {purchase_date}\n"
        f"Current Value: {current_value}\n"
        f"Usage Type: {usage_type}\n"
        f"Condition: {condition}\n"
        f"Coverage Type: {coverage_type}\n"
        f"Policy Duration: {policy_duration}\n"
        f"Insurance Claim History: {insurance_claim_history}\n"
        f"Gadget Age: {gadget_age}\n"
        f"Risk Assessment Score: {risk_assessment_score}\n"
    )
    print("-"*150)



Score: 11.83582
Customer Location: Ontario
Gadget Model: Ipad Pro 13
Product Description: Apple iPad Pro 13 with 13-inch XDR display, sleek design, black color, 8GB RAM, 512GB storage
Purchase Price: 676.82
Purchase Date: 2016-11-09
Current Value: 564.58
Usage Type: Personal
Condition: New
Coverage Type: Comprehensive
Policy Duration: 6
Insurance Claim History: 0
Gadget Age: 8
Risk Assessment Score: 0.92

------------------------------------------------------------------------------------------------------------------------------------------------------

Score: 11.83582
Customer Location: Ontario
Gadget Model: Ipad Pro 13
Product Description: Apple iPad Pro 13 with 13-inch XDR display, sleek design, black color, 8GB RAM, 512GB storage
Purchase Price: 1649.86
Purchase Date: 2016-07-09
Current Value: 931.76
Usage Type: Business
Condition: Refurbished
Coverage Type: Comprehensive
Policy Duration: 3
Insurance Claim History: 4
Gadget Age: 8
Risk Assessment Score: 0.38

--------------------

<ipython-input-7-300dcfd1a8f0>:5: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  response = es.search(
<ipython-input-7-300dcfd1a8f0>:5: ElasticsearchWarning: text_expansion is deprecated. Use sparse_vector instead.
  response = es.search(


## Hybrid Search (BM25 + Semantic Search) Using Insurance Data

### 1. Hybrid Searching with all-mpnet-base-v2 BERT model

In [ ]:
# Define the input keyword
input_keyword = "apple 13"

# Compute the query vector locally
query_vector = model.encode(input_keyword)

# Perform hybrid search: BM25 + k-NN
response = es.search(
    index="insurance_mpnet_data",  # Updated index name
    size=5,  # Number of results to return
    body={
        "query": {
            "bool": {
                "should": [
                    {
                        "match": {
                            "combined_text": {           # Targeted Column
                                "query": input_keyword,  # BM25 match query
                                "boost": 2.0,  # Boost for BM25 relevance
                            }
                        }
                    }
                ]
            }
        },
        "knn": {
            "field": "ml.embeddings",  # Field storing k-NN embeddings
            "k": 10,  # Number of nearest neighbors to return
            "num_candidates": 500,  # Number of candidates to consider per shard
            "query_vector": query_vector.tolist(),  # Use the locally computed query vector
        },
        "rank": {
            "rrf": {  # Reciprocal rank fusion (RRF)
                "rank_window_size": 500,
                "rank_constant": 60,
            }
        },
    },
)

# Process and print results
for hit in response["hits"]["hits"]:
    rank = hit.get("_rank", "N/A")  # Rank from the RRF scoring
    score = hit["_score"]  # Combined score

    # Extract the required fields from the hit
    customer_location = hit["_source"].get("Customer Location: State", "N/A")
    gadget_model = hit["_source"].get("Gadget Model", "N/A")
    product_description = hit["_source"].get("Product Description", "N/A")
    purchase_price = hit["_source"].get("Purchase Price", "N/A")
    purchase_date = hit["_source"].get("Purchase Date", "N/A")
    current_value = hit["_source"].get("Current Value", "N/A")
    usage_type = hit["_source"].get("Usage Type", "N/A")
    condition = hit["_source"].get("Condition", "N/A")
    coverage_type = hit["_source"].get("Coverage Type", "N/A")
    policy_duration = hit["_source"].get("Policy Duration", "N/A")
    insurance_claim_history = hit["_source"].get("Insurance Claim History", "N/A")
    gadget_age = hit["_source"].get("Gadget Age", "N/A")
    risk_assessment_score = hit["_source"].get("Risk Assessment Score", "N/A")

    print(
        f"\nRank: {rank}\nScore: {score}\n"
        f"Customer Location: {customer_location}\n"
        f"Gadget Model: {gadget_model}\n"
        f"Product Description: {product_description}\n"
        f"Purchase Price: {purchase_price}\n"
        f"Purchase Date: {purchase_date}\n"
        f"Current Value: {current_value}\n"
        f"Usage Type: {usage_type}\n"
        f"Condition: {condition}\n"
        f"Coverage Type: {coverage_type}\n"
        f"Policy Duration: {policy_duration}\n"
        f"Insurance Claim History: {insurance_claim_history}\n"
        f"Gadget Age: {gadget_age}\n"
        f"Risk Assessment Score: {risk_assessment_score}\n"
    )
    print("-"*150)



Rank: 1
Score: 0.022722555
Customer Location: Texas
Gadget Model: Ipad Air 13
Product Description: Apple iPad Air 13 with 13-inch Retina display, stylish design, black color, 8GB RAM, 128GB storage
Purchase Price: 3483.35
Purchase Date: 2016-06-02
Current Value: 1347.1
Usage Type: Personal
Condition: Refurbished
Coverage Type: Comprehensive
Policy Duration: 12
Insurance Claim History: 3
Gadget Age: 8
Risk Assessment Score: 0.97

------------------------------------------------------------------------------------------------------------------------------------------------------

Rank: 2
Score: 0.0200506
Customer Location: Texas
Gadget Model: Ipad Air 13
Product Description: Apple iPad Air 13 with 13-inch Retina display, stylish design, black color, 8GB RAM, 128GB storage
Purchase Price: 4001.72
Purchase Date: 2019-04-21
Current Value: 2394.99
Usage Type: Personal
Condition: Damaged
Coverage Type: Comprehensive
Policy Duration: 10
Insurance Claim History: 3
Gadget Age: 5
Risk Assessment

<ipython-input-8-77d2aa80ae83>:8: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  response = es.search(
<ipython-input-8-77d2aa80ae83>:8: ElasticsearchWarning: Deprecated field [rank] used, replaced by [retriever]
  response = es.search(


### 2. Hybrid Searching with elser model

In [ ]:
# Input keyword for the search
input_keyword = "Find all LG AC models with a white color"



# Hybrid search query: BM25 + ELSER model embeddings
response = es.search(
    index="insurance-elser2-data",  # Target index name
    size=5,  # Number of results to return
    body={
        "query": {
            "bool": {
                "should": [
                    {
                        # BM25 match query
                        "match": {
                            "combined_text": {  # Targeted field for BM25 matching
                                "query": input_keyword,
                                "boost": 2.0  # Boosting BM25 relevance
                            }
                        }
                    },
                    {
                        # ELSER text expansion query
                        "text_expansion": {
                            "combined_text_embedding": {
                                "model_id": ".elser_model_2",  # Model ID for ELSER
                                "model_text": input_keyword  # User query text
                            }
                        }
                    }
                ]
            }
        }
    }
)

# Process and print the search results
for hit in response["hits"]["hits"]:
    score = hit["_score"]

    # Extract fields from the hit
    customer_location = hit["_source"].get("Customer Location: State", "N/A")
    gadget_model = hit["_source"].get("Gadget Model", "N/A")
    product_description = hit["_source"].get("Product Description", "N/A")
    purchase_price = hit["_source"].get("Purchase Price", "N/A")
    purchase_date = hit["_source"].get("Purchase Date", "N/A")
    current_value = hit["_source"].get("Current Value", "N/A")
    usage_type = hit["_source"].get("Usage Type", "N/A")
    condition = hit["_source"].get("Condition", "N/A")
    coverage_type = hit["_source"].get("Coverage Type", "N/A")
    policy_duration = hit["_source"].get("Policy Duration", "N/A")
    insurance_claim_history = hit["_source"].get("Insurance Claim History", "N/A")
    gadget_age = hit["_source"].get("Gadget Age", "N/A")
    risk_assessment_score = hit["_source"].get("Risk Assessment Score", "N/A")

    # Print results
    print(
        f"\nScore: {score}\n"
        f"Customer Location: {customer_location}\n"
        f"Gadget Model: {gadget_model}\n"
        f"Product Description: {product_description}\n"
        f"Purchase Price: {purchase_price}\n"
        f"Purchase Date: {purchase_date}\n"
        f"Current Value: {current_value}\n"
        f"Usage Type: {usage_type}\n"
        f"Condition: {condition}\n"
        f"Coverage Type: {coverage_type}\n"
        f"Policy Duration: {policy_duration}\n"
        f"Insurance Claim History: {insurance_claim_history}\n"
        f"Gadget Age: {gadget_age}\n"
        f"Risk Assessment Score: {risk_assessment_score}\n"
    )


<ipython-input-9-2a260a57dfad>:7: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  response = es.search(



Score: 36.499783
Customer Location: California
Gadget Model: 1.5 Ton
Product Description: LG AC 1.5 Ton with smart cooling feature, digital light, white color, advanced air purification
Purchase Price: 995.55
Purchase Date: 2017-07-09
Current Value: 380.62
Usage Type: Personal
Condition: New
Coverage Type: Comprehensive
Policy Duration: 11
Insurance Claim History: 3
Gadget Age: 7
Risk Assessment Score: 0.23


Score: 36.499783
Customer Location: California
Gadget Model: 1.5 Ton
Product Description: LG AC 1.5 Ton with smart cooling feature, digital light, white color, advanced air purification
Purchase Price: 4214.1
Purchase Date: 2018-07-26
Current Value: 2482.56
Usage Type: Business
Condition: New
Coverage Type: Comprehensive
Policy Duration: 12
Insurance Claim History: 0
Gadget Age: 6
Risk Assessment Score: 0.21


Score: 36.499783
Customer Location: California
Gadget Model: 1.5 Ton
Product Description: LG AC 1.5 Ton with smart cooling feature, digital light, white color, advanced air

<ipython-input-9-2a260a57dfad>:7: ElasticsearchWarning: text_expansion is deprecated. Use sparse_vector instead.
  response = es.search(


# 3. Croma Data

## Semantic Serching BERT-mpnet-base-v2 VS elser-model-2

### 1. BERT-mpnet-base-v2 Model

In [ ]:
# Define the input keyword
input_keyword = "apple 16"



# Compute the query vector locally using the BERT model
query_vector = model.encode(input_keyword)

# Perform the search with the locally computed query vector
response = es.search(
    index="croma-mpnet-data",  # Replace with your index name
    size=5,
    body={
        "knn": {
            "field": "ml.embeddings",  # Replace with your field for embeddings
            "k": 10,  # Number of nearest neighbors to return
            "num_candidates": 100,  # Number of nearest neighbor candidates to consider per shard
            "query_vector": query_vector.tolist()  # Use the locally computed query vector (convert to list)
        }
    }
)

# Process and display search results
for hit in response["hits"]["hits"]:
    score = hit["_score"]
    product = hit["_source"].get("name", "N/A")
    category = hit["_source"].get("category", "N/A")
    features = hit["_source"].get("features", "N/A")
    overview = hit["_source"].get("overview", "N/A")
    specification = hit["_source"].get("specification", "N/A")

    print(f"\nScore: {score}")
    print(f"Product: {product}")
    print(f"Category: {category}")
    print(f"Features: {features}")
    print(f"Overview: {overview}")
    print(f"Specification: {specification}")
    print("-"*150)



Score: 0.7373307
Product: Blynk Refurbished Apple iPhone 8 (Space Grey, 64 GB, 2 GB RAM) 
Category: Televisions & Accessories
Features: 11.9 cm (4.7"), 1334 x 750 pixels  2 GB RAM | 64 GB ROM | iOS 13  AppleA11 Bionic chip  Hexa Core processorSingle Rear Camera  | Single Front Camera  1812  mAh Lithium Ion BatterySplash Proof | Water Resistant | Dust Resistant  | Anti Fingerprint  
Overview: No one can resist owning an iPhone whether it is a new or a refurbished option. therefore get the Blynk Refurbished Apple iPhone 8 and explore the world and experience cutting edge technology with the stylish and power-packed Apple iPhone 8 and feel the difference. With 18-point quality checks provided by technical experts and professionals at Blynk, you get an iPhone 8 you experience the best of tech with almost invisible wear and tear. The 11.93 cm (4.7 inches) full HD Retina display with space grey exterior makes your phone stylish and efficient; while the engineered water and dustproof design 

<ipython-input-10-8b6dce733f10>:10: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  response = es.search(


### 2.  elser-model-2 Model

In [ ]:
# Example search query from the user
user_query = "apple xr"



# Function to search for products by name
def search_product_by_name(query: str):
    response = es.search(
        index="croma-elser2-data",  # Your index name
        size=5,  # Number of results to return
        query={
            "text_expansion": {
                "overview_embedding": {
                    "model_id": ".elser_model_2",  # Your model ID
                    "model_text": query  # Search query provided by the user
                }
            }
        }
    )

    # Parse and print the search results
    for hit in response["hits"]["hits"]:
        product_id = hit["_id"]
        score = hit["_score"]
        name = hit["_source"]["name"]
        overview = hit["_source"]["overview"]

        print(f"Score: {score}")
        print(f"Product Name: {name}")
        print(f"Overview: {overview}\n")
        print("-"*150)

search_product_by_name(user_query)


Score: 14.7903
Product Name: Apple iPhone XR (64GB ROM, 3GB RAM, MRY42HN/A, Black) 
Overview: Widen your vision with the stunning and spectacular Apple iPhone XR which flaunts an amazing display, brilliant camera and powerful processor. Available in vibrant classic colours it flaunts a sleek and thin design which is easy to hold and is crafted for perfect grip. Its 2942 mAh battery is perfect for continuous working as long as you don’t give up, because it certainly won’t. It comes with a number of sensors like Face ID, Barometer, Three-axis gyro, Accelerometer, Proximity sensor, Ambient light sensor which help it to stand in style. It has a storage of 64 GB that lets you store files in ease. It runs on iOS 13.
Enhances Efficiency
The Apple iPhone XR uses Hexa-core A12 Bionic Chip Processor along with Second-generation Neural Engine, so you can play hi-end games or run heavy files and carry out tasks without worrying about the performance. The upgraded CPU ensures a seamless response ti

<ipython-input-11-a4e90a91fd11>:8: ElasticsearchWarning: text_expansion is deprecated. Use sparse_vector instead.
  response = es.search(


## Hybrid Search (BM25 + Semantic Search) Using Croma Data

### 1. Hybrid Searching with all-mpnet-base-v2 BERT model

In [ ]:
# Define the input keyword
input_keyword = "macbook wind in mumbai"

# Compute the query vector locally using the BERT model
query_vector = model.encode(input_keyword)

# Perform hybrid search: BM25 + k-NN
response = es.search(
    index="croma-mpnet-data",  # Target index name
    size=5,  # Number of results to return
    body={
        "query": {
            "bool": {
                "should": [
                    {
                        "match": {
                            "combined_text": {  # Targeted field for BM25 matching
                                "query": input_keyword,  # BM25 match query
                                "boost": 2.0,  # Boosting BM25 relevance
                            }
                        }
                    }
                ]
            }
        },
        "knn": {
            "field": "ml.embeddings",  # Field storing k-NN embeddings
            "k": 10,  # Number of nearest neighbors to return
            "num_candidates": 100,  # Number of candidates to consider per shard
            "query_vector": query_vector.tolist(),  # Use the locally computed query vector
        },
        "rank": {
            "rrf": {  # Reciprocal Rank Fusion (RRF) for ranking combined results
                "rank_window_size": 500,
                "rank_constant": 60,
            }
        },
    },
)

# Process and display search results
for hit in response["hits"]["hits"]:
    rank = hit.get("_rank", "N/A")  # Rank from RRF scoring
    score = hit["_score"]  # Combined score

    # Extract fields from the hit
    product = hit["_source"].get("name", "N/A")
    category = hit["_source"].get("category", "N/A")
    features = hit["_source"].get("features", "N/A")
    overview = hit["_source"].get("overview", "N/A")
    specification = hit["_source"].get("specification", "N/A")

    # Print results
    print(
        f"\nRank: {rank}\nScore: {score}\n"
        f"Product: {product}\n"
        f"Category: {category}\n"
        f"Features: {features}\n"
        f"Overview: {overview}\n"
        f"Specification: {specification}\n"
        + "-" * 150
    )


<ipython-input-12-d9a75ef43494>:8: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  response = es.search(



Rank: 1
Score: 0.016393442
Product: Apple MacBook Air (MGN63HN/A) M1 Chip macOS Big Sur Laptop (8GB RAM, 256GB SSD, Apple M1 GPU, 33.78cm, Space Grey) 
Category: Televisions & Accessories
Features:  33.78 cm (13.3"), Retina Apple M1 Chip RAM: 8GB, ROM: 256GB SSD Apple M1 GPU macOS Big Sur Best Suited For: Everyday Use | Office Use | Home | Students | Travel 
Overview: Why buy Apple MacBook Air (MGN63HN/A) M1 Chip macOS Big Sur Laptop

256GB SSD helps you to store all your files, videos, pictures, games, etc
Apple M1 Chip Processor enables you to use it with ease
13.3-inch Retina Display enables you to view every contents with perfect clarity
8GB RAM Performance empowers you to do multitasking without any fear of lagging and works smoothly

Use the Apple MacBook Air (MGN63HN/A) M1 Chip macOS Big Sur Laptop has set standards in computing thanks to its Apple M1 chip Processor. It has 8-Core CPU with 4 perform­ance cores and 4 efficiency cores, 8-core GPU, 16-core Neural Engine. The proce

<ipython-input-12-d9a75ef43494>:8: ElasticsearchWarning: Deprecated field [rank] used, replaced by [retriever]
  response = es.search(


### 2. Hybrid Searching with elser model

In [ ]:
# Example search query from the user
user_query = "apple xr"

# Function to perform hybrid search: BM25 + ELSER embeddings
def hybrid_search(query: str):
    response = es.search(
        index="croma-elser2-data",  # Your index name
        size=5,  # Number of results to return
        body={
            "query": {
                "bool": {
                    "should": [
                        {
                            # BM25 match query
                            "match": {
                                "text_sparse": {  # Field for BM25 matching
                                    "query": query,
                                    "boost": 2.0  # Boosting BM25 relevance
                                }
                            }
                        },
                        {
                            # ELSER text expansion query
                            "text_expansion": {
                                "overview_embedding": {  # Field for ELSER embeddings
                                    "model_id": ".elser_model_2",  # Model ID for ELSER
                                    "model_text": query  # User query text
                                }
                            }
                        }
                    ]
                }
            }
        }
    )

    # Parse and print the search results
    for hit in response["hits"]["hits"]:
        product_id = hit["_id"]
        score = hit["_score"]
        name = hit["_source"]["name"]
        overview = hit["_source"]["overview"]

        print(f"Score: {score}")
        print(f"Product Name: {name}")
        print(f"Overview: {overview}\n")
        print("-" * 150)

# Perform hybrid search with the user's query
hybrid_search(user_query)


<ipython-input-13-49eda635a1a6>:6: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  response = es.search(


Score: 14.7903
Product Name: Apple iPhone XR (64GB ROM, 3GB RAM, MRY42HN/A, Black) 
Overview: Widen your vision with the stunning and spectacular Apple iPhone XR which flaunts an amazing display, brilliant camera and powerful processor. Available in vibrant classic colours it flaunts a sleek and thin design which is easy to hold and is crafted for perfect grip. Its 2942 mAh battery is perfect for continuous working as long as you don’t give up, because it certainly won’t. It comes with a number of sensors like Face ID, Barometer, Three-axis gyro, Accelerometer, Proximity sensor, Ambient light sensor which help it to stand in style. It has a storage of 64 GB that lets you store files in ease. It runs on iOS 13.
Enhances Efficiency
The Apple iPhone XR uses Hexa-core A12 Bionic Chip Processor along with Second-generation Neural Engine, so you can play hi-end games or run heavy files and carry out tasks without worrying about the performance. The upgraded CPU ensures a seamless response ti

<ipython-input-13-49eda635a1a6>:6: ElasticsearchWarning: text_expansion is deprecated. Use sparse_vector instead.
  response = es.search(


# Gradio App

## 1. Insurance Data

In [4]:
import gradio as gr
from sentence_transformers import SentenceTransformer
from elasticsearch import Elasticsearch

# Initialize Elasticsearch connection
CLOUD_ID = "20350d848e2d40d796d363d6bd9487d8:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvOjQ0MyRhZDcyYWQ3YTdlZGM0NjdjYjFhMmM4NjQxNWIwMmZmMSQ3ZWM2ODJjMGE4YTM0MjY0YTY1ODBjYWU4MjY0M2IwOQ=="
es = Elasticsearch(cloud_id=CLOUD_ID, basic_auth=("elastic", "FyjsNg9xN6LnAo0yFj3gEDTk"), request_timeout=3600)

# Initialize the BERT model
model = SentenceTransformer("all-mpnet-base-v2", device="cpu")

# Function for BERT model search
def bert_search(input_keyword):
    query_vector = model.encode(input_keyword)

    response = es.search(
        index="insurance_mpnet_data",  # Updated index name
        size=5,
        knn={
            "field": "ml.embeddings",  # Field storing precomputed embeddings
            "k": 10,  # Number of nearest neighbors to return as top hits.
            "num_candidates": 500,  # Number of nearest neighbor candidates to consider per shard.
            "query_vector": query_vector,  # Use the locally computed query vector
        },
    )

    result = []
    for hit in response["hits"]["hits"]:
        score = hit["_score"]
        gadget_model = hit["_source"].get("Gadget Model", "N/A")
        product_description = hit["_source"].get("Product Description", "N/A")
        customer_location = hit["_source"].get("Customer Location: State", "N/A")
        purchase_price = hit["_source"].get("Purchase Price", "N/A")
        policy_duration = hit["_source"].get("Policy Duration", "N/A")


        result.append(
            f"\nScore: {score}\n"
            f"Gadget Model: {gadget_model}\n"
            f"Product Description: {product_description}\n"
            f"Customer Location: {customer_location}\n"
            f"Purchase Price: {purchase_price}\n"
            f"Policy Duration: {policy_duration}\n"
            f"------------------------------------------------------------------------------------------------------------------------"
        )
        ("-"*150)
    return "\n".join(result)

# Function for ELSER model search
def elser_search(input_keyword):
    response = es.search(
        index="insurance-elser2-data",  # Target index name
        size=5,  # Number of documents to retrieve
        body={
            "query": {
                "text_expansion": {
                    "combined_text_embedding": {
                        "model_id": ".elser_model_2",  # Model ID for ELSER
                        "model_text": input_keyword  # User query text
                    }
                }
            }
        }
    )

    result = []
    for hit in response["hits"]["hits"]:
        score = hit["_score"]
        gadget_model = hit["_source"].get("Gadget Model", "N/A")
        product_description = hit["_source"].get("Product Description", "N/A")
        customer_location = hit["_source"].get("Customer Location: State", "N/A")
        purchase_price = hit["_source"].get("Purchase Price", "N/A")
        policy_duration = hit["_source"].get("Policy Duration", "N/A")

        result.append(
            f"\nScore: {score}\n"
            f"Gadget Model: {gadget_model}\n"
            f"Product Description: {product_description}\n"
            f"Customer Location: {customer_location}\n"
            f"Purchase Price: {purchase_price}\n"
            f"Policy Duration: {policy_duration}\n"
            f"------------------------------------------------------------------------------------------------------------------------"
        )
    return "\n".join(result)

# Define Gradio UI
def search_ui(input_keyword):
    bert_result = bert_search(input_keyword)
    elser_result = elser_search(input_keyword)

    return bert_result, elser_result

# Create the Gradio Interface with top query box and side-by-side result layout
with gr.Blocks() as interface:
    input_box = gr.Textbox(label="Enter Search Query")
    submit_button = gr.Button("Submit", elem_id="submit-btn")  # Add elem_id for styling

    with gr.Row():
        textbox1 = gr.Textbox(label="Traditional Semantic Model Results", lines=20)
        textbox2 = gr.Textbox(label="Improved Semantic Model Results", lines=20)

    submit_button.click(search_ui, inputs=input_box, outputs=[textbox1, textbox2])

# Custom CSS to change the submit button color to orange
interface.css = """
    #submit-btn {
        background-color: orange;
        color: white;
        border: none;
    }
    #submit-btn:hover {
        background-color: darkorange;
    }
"""

# Launch the interface
interface.launch()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a37902f236c5ed9c9c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## 2. Croma Data

In [ ]:
import gradio as gr
from sentence_transformers import SentenceTransformer
from elasticsearch import Elasticsearch

# Initialize Elasticsearch connection
CLOUD_ID = "20350d848e2d40d796d363d6bd9487d8:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvOjQ0MyRhZDcyYWQ3YTdlZGM0NjdjYjFhMmM4NjQxNWIwMmZmMSQ3ZWM2ODJjMGE4YTM0MjY0YTY1ODBjYWU4MjY0M2IwOQ=="
es = Elasticsearch(cloud_id=CLOUD_ID, basic_auth=("elastic", "FyjsNg9xN6LnAo0yFj3gEDTk"), request_timeout=3600)

# Initialize the BERT model
model = SentenceTransformer("all-mpnet-base-v2", device="cpu")

# Function for BERT model search (from the second provided code)
def bert_search(input_keyword):
    query_vector = model.encode(input_keyword)

    response = es.search(
        index="croma-mpnet-data",  # Updated index name
        size=5,
        body={
            "knn": {
                "field": "ml.embeddings",  # Field storing precomputed embeddings
                "k": 10,  # Number of nearest neighbors to return
                "num_candidates": 100,  # Number of nearest neighbor candidates to consider per shard
                "query_vector": query_vector.tolist(),  # Convert query vector to list
            }
        }
    )

    result = []
    for hit in response["hits"]["hits"]:
        score = hit["_score"]
        product = hit["_source"].get("name", "N/A")
        category = hit["_source"].get("category", "N/A")
        features = hit["_source"].get("features", "N/A")

        result.append(
            f"\nScore: {score}\n"
            f"Product: {product}\n"
            f"Category: {category}\n"
            f"Features: {features}\n"
            f"------------------------------------------------------------------------------------------------------------------------"
        )
    return "\n".join(result)

# Function for ELSER model search (from the second provided code)
def elser_search(input_keyword):
    response = es.search(
        index="croma-elser2-data",  # Target index name
        size=5,  # Number of documents to retrieve
        body={
            "query": {
                "text_expansion": {
                    "overview_embedding": {
                        "model_id": ".elser_model_2",  # Model ID for ELSER
                        "model_text": input_keyword  # User query text
                    }
                }
            }
        }
    )

    result = []
    for hit in response["hits"]["hits"]:
        score = hit["_score"]
        product = hit["_source"].get("name", "N/A")
        category = hit["_source"].get("category", "N/A")
        features = hit["_source"].get("features", "N/A")

        result.append(
            f"\nScore: {score}\n"
            f"Product: {product}\n"
            f"Category: {category}\n"
            f"Features: {features}\n"
            f"------------------------------------------------------------------------------------------------------------------------"
        )
    return "\n".join(result)

# Define Gradio UI function to handle both searches
def search_ui(input_keyword):
    bert_result = bert_search(input_keyword)
    elser_result = elser_search(input_keyword)

    return bert_result, elser_result

# Create the Gradio Interface with top query box and side-by-side result layout
with gr.Blocks() as interface:
    input_box = gr.Textbox(label="Enter Search Query")
    submit_button = gr.Button("Submit", elem_id="submit-btn")  # Add elem_id for styling

    with gr.Row():
        textbox1 = gr.Textbox(label="Traditional Semantic Model Results", lines=20)
        textbox2 = gr.Textbox(label="Improved Semantic Model Results", lines=20)

    submit_button.click(search_ui, inputs=input_box, outputs=[textbox1, textbox2])

# Custom CSS to change the submit button color to orange
interface.css = """
    #submit-btn {
        background-color: orange;
        color: white;
        border: none;
    }
    #submit-btn:hover {
        background-color: darkorange;
    }
"""

# Launch the interface
interface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fb3d08fe828d9bcd07.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
